# Custom Embeddings
LlamaIndex supports embeddings from OpenAI, Azure, and Langchain. But if this isn't enough, you can also implement any embeddings model!

The example below uses Instructor Embeddings ([install/setup details here](https://huggingface.co/hkunlp/instructor-large)), and implements a custom embeddings class. Instructor embeddings work by providing text, as well as "instructions" on the domain of the text to embed. This is helpful when embedding text from a very specific and specialized topic.


In [ ]:
# Install dependencies
# !pip install InstructorEmbedding torch transformers sentence-transformers

In [23]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]

## Custom Embeddings Implementation

In [29]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR
from llama_index.embeddings.base import BaseEmbedding


class InstructorEmbeddings(BaseEmbedding):
    def __init__(
        self,
        instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent a document for semantic search:",
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name)
        self._instruction = instruction
        super().__init__(**kwargs)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, query]])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, text]])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode([[self._instruction, text] for text in texts])
        return embeddings

## Usage Example

In [30]:
from llama_index import ServiceContext, SimpleDirectoryReader, VectorStoreIndex

In [31]:
documents = SimpleDirectoryReader("../../data/paul_graham/").load_data()

In [32]:
service_context = ServiceContext.from_defaults(
    embed_model=InstructorEmbeddings(embed_batch_size=2), chunk_size=512
)

# if running for the first time, will download model weights first!
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

load INSTRUCTOR_Transformer


max_seq_length  512


In [33]:
response = index.as_query_engine().query("What did the author do growing up?")
print(response)


The author grew up writing short stories, programming on an IBM 1401, writing essays on various topics, working on spam filters, painting, cooking for groups, and buying a building in Cambridge.
